# Adding a transition matrix and Returns to E

In [13]:
import numpy as np
import pandas as pd


data_frame = pd.read_csv('data.respy.dat', delim_whitespace=True, header=0,
        na_values='.')
data_frame.set_index(['Identifier', 'Period'], drop=False, inplace=True)


In [14]:
data_frame.describe()

,Identifier,Period,Choice,Wage,Experience_A,Experience_B,Years_Schooling,Lagged_Schooling,Total_Reward_1,Total_Reward_2,...,Total_Reward_4,Systematic_Reward_1,Systematic_Reward_2,Systematic_Reward_3,Systematic_Reward_4,Shock_Reward_1,Shock_Reward_2,Shock_Reward_3,Shock_Reward_4,Discount_Rate
count,40.000000,40.000000,40.000000,6.000000,40.0,40.000000,40.000000,40.000000,40.000000,40.000000,...,40.000000,40.000000,40.000000,40.000000,4.000000e+01,40.000000,40.000000,40.000000,40.000000,40.0
mean,4.500000,1.500000,3.225000,42973.701667,0.0,0.125000,11.200000,0.725000,5550.616685,14547.518662,...,26225.728558,3715.451494,8408.780167,18864.144996,2.622546e+04,1.465798,1.718994,-0.312124,0.267384,0.0
std,2.908872,1.132277,0.697523,12183.294681,0.0,0.334932,0.822753,0.452203,6290.641644,14559.194989,...,0.691865,479.891966,1175.929636,10542.356035,3.684324e-12,1.642446,1.686720,0.840408,0.691865,0.0
min,0.000000,0.000000,2.000000,27770.720000,0.0,0.000000,10.000000,0.000000,252.641601,983.496283,...,26224.282731,3127.511322,7204.087624,3907.995855,2.622546e+04,0.073582,0.136519,-3.041458,-1.178444,0.0
25%,2.000000,0.750000,3.000000,35491.502500,0.0,0.000000,10.750000,0.000000,1789.656406,3991.936286,...,26225.220532,3356.971015,7376.500935,9946.737150,2.622546e+04,0.565883,0.472807,-0.757810,-0.240642,0.0
50%,4.500000,1.500000,3.000000,40393.005000,0.0,0.000000,11.000000,1.000000,3003.399793,8212.398947,...,26225.804662,3469.063762,7842.503563,21312.337438,2.622546e+04,0.859282,1.139964,-0.254836,0.343488,0.0
75%,7.000000,2.250000,4.000000,52361.750000,0.0,0.000000,12.000000,1.000000,5254.563042,20555.822468,...,26226.164280,4207.623527,9463.781424,28716.208086,2.622546e+04,1.530399,2.121477,0.111729,0.703106,0.0
max,9.000000,3.000000,4.000000,59076.480000,0.0,1.000000,12.000000,1.000000,22993.501445,59076.476595,...,26227.567858,4294.892646,10503.939505,28716.208086,2.622546e+04,7.352012,7.467575,1.283375,2.106683,0.0


In [19]:
# Create a new column with the next period_s choice
df = data_frame
df['Choice_Next'] = np.nan
df['Choice_Next'] = df.groupby(level='Identifier')['Choice'].shift(-1)

In [30]:
df[['Choice', 'Choice_Next']].head(10)

tb = pd.crosstab(df['Choice'], df['Choice_Next'], normalize=True)


In [38]:
cat_choice = pd.Categorical(df['Choice'], categories=range(1, 5))
cat_choice_next = pd.Categorical(df['Choice_Next'], categories=range(1, 5))
pd.crosstab(cat_choice, cat_choice_next, margins=True)

col_0,1,2,3,4,All
row_0,,,,,
1,0,0,0,0,0
2,0,0,0,4,4
3,0,4,9,6,19
4,0,2,0,5,7
All,0,6,9,15,30


In [34]:
# I need to be able to deal with missing values ...
tf = np.tile(np.nan, (4, 4))

for i in range(4):
    if i not in tb.columns:
        tf[i, :] = 0.00
    for j in range(4):
        tf = tb